In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D

matplotlib.use('agg')

def generate_gaussian_samples(mu, cov):
    """Generate samples from multiple Gaussian distributions based on specified means and covariance matrices."""
    np.random.seed(1)
    samples = np.zeros((2000*len(mu), len(mu[0])))
    for i, (mean, covariance) in enumerate(zip(mu, cov)):
        samples[i*2000:(i+1)*2000] = np.random.multivariate_normal(mean, covariance, 2000)
    plt.figure(figsize=(8, 8))
    plt.scatter(samples[:, 0], samples[:, 1], alpha=0.2)
    plt.title('Generated Samples')
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.axis('equal')
    return samples

def initialize_parameters(num_components, seed):
    """Initialize the parameters for the EM algorithm: mixing probabilities, means, and covariance matrices."""
    pis = [1 / num_components] * num_components
    np.random.seed(seed)
    mus = np.random.rand(num_components, 2) * 4 - 2  # Randomly initialized means
    sigmas = np.array([np.eye(2) for _ in range(num_components)])  # Identity matrices as initial covariances
    return pis, mus, sigmas

def run_em_algorithm(data, max_iter, pi, mu, cov, threshold=0.1):
    """Iteratively apply E-step and M-step of the EM algorithm and plot intermediate results."""
    previous_aic = float('inf')
    no_improve_count = 0
    aic_list = []
    
    for it in range(max_iter):
        r, m_c, pi = expectation_step(data, mu, cov, pi)
        mu, cov = maximization_step(data, r, m_c)
        current_aic = compute_aic(data, mu, cov, pi)
        aic_list.append(current_aic)
        plot_gaussian_mixtures(data, mu, cov, pi, it)
        
        if it % 10 == 0  or it == max_iter - 1:
            print(f"Iteration {it}: AIC={current_aic}, pi={pi}, \nmu=\n{mu}, \ncov=\n{cov}")
        if previous_aic - current_aic < threshold:
            no_improve_count += 1
            if no_improve_count >= 3:
                print(f"Terminating early: AIC improvement less than {threshold} for three consecutive iterations.")
                return aic_list
        else:
            no_improve_count = 0
        
        previous_aic = current_aic
    print(f'Algorithm did not converge in {max_iter} iterations')
    return aic_list

def expectation_step(data, mu, cov, pi):
    """E-step: Calculate responsibilities based on current parameter estimates."""
    r = np.array([pi[i] * multivariate_normal(mu[i], cov[i]).pdf(data) for i in range(len(pi))]).T
    r /= r.sum(axis=1, keepdims=True)
    m_c = r.sum(axis=0)
    pi = m_c / m_c.sum()
    return r, m_c, pi

def maximization_step(data, r, m_c):
    """M-step: Update parameters based on current responsibilities."""
    mu = np.array([r[:, i].dot(data) / m_c[i] for i in range(len(m_c))])
    cov = np.array([(np.dot((data - mu[i]).T, (data - mu[i]) * r[:, i][:, np.newaxis]) / m_c[i]) for i in range(len(m_c))])
    return mu, cov

def compute_aic(data, mu, cov, pi):
    """Compute the Akaike Information Criterion for the model."""
    k = len(mu) * (len(mu[0]) + len(mu[0]) * (len(mu[0]) + 1) / 2 + 1)  # Number of parameters
    likelihood = np.sum(np.log(np.sum([pi[i] * multivariate_normal(mu[i], cov[i]).pdf(data) for i in range(len(pi))], axis=0)))
    aic = 2 * k - 2 * likelihood
    return aic

def plot_gaussian_mixtures(data, mu, cov, pi, it):
    """Visualize the data along with the Gaussian mixture model probability contours."""
    x, y = np.meshgrid(np.linspace(-4, 4, 100), np.linspace(-4, 4, 100))
    pos = np.dstack((x, y))
    
    fig, ax = plt.subplots(figsize=(10, 7))
    
    # Calculate the total density for the mixture
    total_density = np.zeros(pos.shape[:2])
    for i in range(len(pi)):
        rv = multivariate_normal(mu[i], cov[i])
        total_density += pi[i] * rv.pdf(pos)

    # Plot contours for the total density
    contour = ax.contourf(x, y, total_density, levels=8)
    cbar = plt.colorbar(contour, ax=ax)
    cbar.set_label('Probability Density')

    # Scatter plot of the samples
    ax.scatter(data[:, 0], data[:, 1], alpha=0.02, color='red', marker='o', label='Data points')
    ax.set_xlabel('X1')
    ax.set_ylabel('X2')
    ax.set_title(f'Gaussian Mixture Model Fit - Contour Plot {it}')
    
    fig.savefig(f'2.5d_images/{it+1}.png')
    plt.close(fig)
    

def main():
    n_modules = 4
    seed = 0
    max_iterations = 200
    
    mu = [[-0.9, -0.9], [-0.9, 0.9], [0.9, -0.9], [0.4, 0.4]]
    cov = [[[0.55, -0.18], [-0.18, 0.35]], [[0.4, 0.15], [0.15, 0.4]], [[0.38, 0.18], [0.18, 0.39]], [[0.43, 0.3], [0.3, 0.43]]]
    samples = generate_gaussian_samples(mu, cov)
    pis, mus, sigmas = initialize_parameters(n_modules, seed)
    aic_list = run_em_algorithm(samples, max_iterations, pis, mus, sigmas, threshold=0.1)
    
    fig = plt.figure(figsize=(12, 5))
    ax0 = fig.add_subplot(111)
    ax0.plot(aic_list)
    ax0.set_ylabel('AIC value')
    fig.savefig('2.5d_images/AIC.png')

if __name__ == '__main__':
    main()

Iteration 0: AIC=46068.87992757093, pi=[0.21869776 0.34930566 0.2987562  0.13324038], 
mu=
[[ 0.00183385  0.08447452]
 [ 0.20803113 -0.5337745 ]
 [-0.50407659 -0.16983935]
 [-0.41943975  0.68186497]], 
cov=
[[[ 1.01799023  0.01531956]
  [ 0.01531956  0.9342032 ]]

 [[ 0.93273141  0.01102872]
  [ 0.01102872  0.84787876]]

 [[ 0.97966242 -0.01422195]
  [-0.01422195  0.88784757]]

 [[ 0.96152768  0.06807923]
  [ 0.06807923  0.77942363]]]
Iteration 10: AIC=45800.693051034345, pi=[0.20515351 0.34462498 0.2853974  0.1648241 ], 
mu=
[[ 0.00976468  0.17273807]
 [ 0.35367462 -0.74820857]
 [-0.66326925 -0.19215253]
 [-0.41260866  0.90635227]], 
cov=
[[[ 1.07867284  0.11974921]
  [ 0.11974921  0.95144165]]

 [[ 0.77099626  0.08422686]
  [ 0.08422686  0.58477419]]

 [[ 0.85434078 -0.00801602]
  [-0.00801602  0.82793441]]

 [[ 0.9473081   0.15501606]
  [ 0.15501606  0.38363988]]]
Iteration 20: AIC=45475.54232937662, pi=[0.19456472 0.32954262 0.29016054 0.18573213], 
mu=
[[ 0.05185783  0.30599488]
 

In [2]:
import imageio.v2 as imageio
import os

def create_gif(image_folder, output_path, duration, last_frame_duration, max_iterations):
    """Create a GIF from a set of images in a folder with the last frame having a longer duration."""
    images = []
    existing_files = os.listdir(image_folder)
    for i in range(max_iterations):
        if f'{i}.png' in existing_files:
            file_path = os.path.join(image_folder, f'{i}.png')
            images.append(imageio.imread(file_path))
    # Append the last image additional times to extend its display time
    last_image = images[-1]
    extended_frames = int(last_frame_duration / duration)
    images.extend([last_image] * extended_frames)
    
    imageio.mimsave(output_path, images, duration=duration)

def main():
    image_folder = '2.5d_images'  # Folder containing images
    output_path = '2.5d_gmm.gif'  # Desired output GIF file path
    frame_duration = 0.05  # Duration of each frame in the GIF
    last_frame_duration = 0.5  # Extended duration for the last frame
    max_iterations = 200

    create_gif(image_folder, output_path, frame_duration, last_frame_duration, max_iterations)

if __name__ == "__main__":
    main()


In [3]:
import cv2
import os

def create_video(image_folder, output_path, frame_duration, max_iterations):
    """Create a video from a set of images in a folder."""
    images = []
    existing_files = os.listdir(image_folder)
    for i in range(max_iterations):
        if f'{i}.png' in existing_files:
            images.append(f'{i}.png')  # Change this to match your image format
    # Determine the width and height from the first image
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID'
    video = cv2.VideoWriter(output_path, fourcc, 1 / frame_duration, (width, height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    # Extend the last frame duration
    for _ in range(int(0.5 / frame_duration)):  # Adjust '0.5' to however many seconds you want the last frame to hold
        video.write(frame)

    cv2.destroyAllWindows()
    video.release()

def main():
    image_folder = '2.5d_images'  # Folder containing images
    output_path = '2.5d_gmm.mp4'  # Desired output video file path
    frame_duration = 0.05  # Duration of each frame in the video
    max_iterations = 200

    create_video(image_folder, output_path, frame_duration, max_iterations)

if __name__ == "__main__":
    main()
